In [16]:
import numpy as np

R = np.array([[0.5076271,  0.3434859,  0.7901469],
  [ 0.8382091,  0.0152542, -0.5451356],
  [-0.1992995,  0.9390339, -0.2801695]])

if (R[2,2] < 0):
    if (R[0,0] >R[1,1]):
        t = 1 + R[0,0] -R[1,1] -R[2,2]
        q = np.array([t, R[0,1]+R[1,0], R[2,0]+R[0,2], R[1,2]-R[2,1]])
    else:
        t = 1 -R[0,0] + R[1,1] -R[2,2]
        q = np.array([R[0,1]+R[1,0], t, R[1,2]+R[2,1], R[2,0]-R[0,2]])
else:
    if (R[0,0] < -R[1,1]):
        t = 1 -R[0,0] -R[1,1] + R[2,2]
        q = np.array([R[2,0]+R[0,2], R[1,2]+R[2,1], t, R[0,1]-R[1,0]])
    else:
        t = 1 + R[0,0] + R[1,1] + R[2,2]
        q = np.array([R[1,2]-R[2,1], R[2,0]-R[0,2], R[0,1]-R[1,0], t])
q *= 0.5 / np.sqrt(t)
q

array([ 0.66568431,  0.44378956,  0.22189474, -0.55738489])

In [17]:
from scipy.spatial.transform import Rotation as ScipyRot
plot_lambda_bar = []
rot = ScipyRot.from_matrix(R)
print(rot.as_quat())

[0.66568432 0.44378957 0.22189475 0.5573849 ]


In [3]:
import numpy as np
b_v = np.random.normal(0, 0.1, 100)
b_v

array([ 0.10489715, -0.06296308, -0.04189654,  0.07257421, -0.09457028,
        0.0972766 ,  0.05213279, -0.18119277, -0.19042857, -0.01521681,
       -0.07999577, -0.02552533, -0.01515034,  0.1825305 ,  0.02036927,
        0.08217705, -0.02604619,  0.26649835,  0.07568474,  0.04959916,
       -0.00045098, -0.03391113, -0.22827685,  0.17474762, -0.18443397,
       -0.12463445,  0.04224442, -0.08907828,  0.03262268, -0.16126957,
       -0.06409194,  0.15149178, -0.01819313, -0.02722286, -0.06038708,
        0.13292175,  0.11277012, -0.00819872, -0.15643975,  0.08452313,
       -0.00317457, -0.12489057, -0.02768956, -0.00841947,  0.07699461,
       -0.0909824 ,  0.17773139,  0.03115058,  0.12465928, -0.03143118,
        0.03898944, -0.1122807 , -0.11031554,  0.06165646,  0.11944616,
        0.1274083 , -0.12425541, -0.049371  ,  0.06852414, -0.32488107,
       -0.00497211, -0.00325366, -0.06194888,  0.11862655, -0.08872255,
       -0.01688986, -0.07763043,  0.09891535, -0.07238144, -0.10